In [1]:
!pip install -q transformers datasets sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
pip install --upgrade numpy datasets transformers

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv").sample(n=2000, random_state=42)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['target'])

In [5]:
train_ds = Dataset.from_pandas(train_df[['question_text', 'target']])
val_ds = Dataset.from_pandas(val_df[['question_text', 'target']])

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def tokenize(batch): return tokenizer(batch["question_text"], padding="max_length", truncation=True, max_length=64)

In [8]:
train_ds = train_ds.map(tokenize, batched=True).rename_column("target", "labels").with_format("torch")
val_ds = val_ds.map(tokenize, batched=True).rename_column("target", "labels").with_format("torch")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": p, "recall": r}


In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    report_to="none"
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-15-3457493100>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

Step,Training Loss
500,0.124300
1000,0.041000


TrainOutput(global_step=1000, training_loss=0.08268313598632812, metrics={'train_runtime': 2051.6668, 'train_samples_per_second': 3.899, 'train_steps_per_second': 0.487, 'total_flos': 65872730112000.0, 'train_loss': 0.08268313598632812, 'epoch': 5.0})

In [17]:
trainer.train()

Step,Training Loss
500,0.051900
1000,0.014900


TrainOutput(global_step=1000, training_loss=0.03339532995223999, metrics={'train_runtime': 2199.3468, 'train_samples_per_second': 3.637, 'train_steps_per_second': 0.455, 'total_flos': 65872730112000.0, 'train_loss': 0.03339532995223999, 'epoch': 5.0})